In [1]:
import numpy as np
from scipy.optimize import fsolve

# Given OIS rates (annualized percentage, converted to decimal)
ois_rates = {
    .5: 0.0025, 1: 0.0030, 2: 0.00325, 3: 0.00335,
    4: 0.0035, 5: 0.0036, 7: 0.0040, 10: 0.0045,
    15: 0.0050, 20: 0.00525, 30: 0.0055
}


In [2]:
def D_0(f):
    return 1/(1+f/360)**180

def D_n(f):
    return 1/(1+f/360)**360

In [3]:
dict_D = {}
dict_f = {}

f_0 = 360*((.5*ois_rates[.5]+1)**(1/180)-1)
dict_f[0.5] = f_0
dict_D[0.5] = D_0(f_0)

Solve for 1 year f and D(0, 1y) by using a fsolver.
1. make function creator for PV fixed
2. make function creator for PV flt
3. put two together in another function
4. use combined function in fsolver

In [4]:
def fn_pv_fix(D, tenor, ois_rate):
    """
    return function of PV fixed interms of f_tenor
    """
    # delta_T = 0
    # if tenor > 1:
    #     delta_T
    # for tenor, DF in D.items():
        # # iterate through all DF in D list and return expression
        # print(tenor, DF)
        # y
    # L = (1+f_0/360)^(tenor*360)
    fn_D = lambda f_1: ois_rate/((1+f_0/360)**(tenor*360)*(1+f_1/360)**(tenor*360))

    return fn_D

fn_1 = fn_pv_fix(dict_D, .5, ois_rates[1])
fn_1(.003488786)

0.0029910326172491834

In [5]:
def fn_pv_flt(dict_D, tenor, ois_rate,f_0):
    """
    Return function of PV float in terms of f_tenor
    """
    fn_D = lambda  f_1: 1 - 1/((1 + f_0 / 360)**(180) * (1 + f_1 / 360)**(180))
    return fn_D

# Example usage (assuming ois_rates["6m"] and D are defined)
fn_2 = fn_pv_flt(dict_D, 0.5, ois_rates[1], f_0)
fn_2(0.003488785)


0.0029891270851050145

In [6]:
eqn = lambda f_1: fn_1(f_1) - fn_2(f_1)

res = fsolve(eqn, 1e-5)[0]
res, fn_1(res)-fn_2(res)

# save result to f and Dict D

dict_f[1] = res
dict_D[1] = dict_D[0.5]*D_0(res)

In [7]:
dict_f

{0.5: 0.0024984474705913584, 1: 0.0034925961006445987}

In [8]:

dict_D

{0.5: 0.9987515605493182, 1: 0.9970089730807411}

In [9]:
def fn_pv_fix(dict_D, tenor, ois_rate):
    """
    return function of PV fixed interms of f_tenor
    """
    # delta_T = 0
    # if tenor > 1:
    #     delta_T
    # for tenor, DF in D.items():
        # # iterate through all DF in D list and return expression
        # print(tenor, DF)
        # y
    # L = (1+f_0/360)^(tenor*360)
    if tenor <2 :
        fn_fix = lambda f_n: ois_rate/((1+f_0/360)**(.5*360)*(1+f_n/360)**(.5*360))
    else:
        sum_df = sum(dict_D.values()) - dict_D[0.5]
        fn_fix = lambda f_n: (sum_df + (1 + f_n / 360)**(360)) * ois_rate
    return fn_fix
fn_fix = fn_pv_fix(dict_D, 2, ois_rates[2])
fn_fix(0.003488785)

0.0065016374605347165

In [10]:
def fn_pv_flt(D, tenor, ois_rate,dict_f):
    """
    Return function of PV float in terms of f_tenor
    """
    sum=0
    product = 1

    if tenor <2 :
        fn_flt = lambda  f_1: 1 - 1/((1 + dict_f[0.5] / 360)**(180) * (1 + f_1 / 360)**(180))
    else:
        fn_flt = lambda f_n: dict_D[1]*((1+dict_f[0.5]/360)**180*(1+dict_f[1]/360)**180-1) \
                            + (dict_D[1] / (1+f_n/360)**360)*((1+f_n/360)**360-1)
            
    return fn_flt

# Example usage (assuming ois_rates["6m"] and D are defined)
fn_flt = fn_pv_flt(dict_D, 2, ois_rates[2], dict_f)
fn_flt(0.003488785)


0.006463299516241901

In [11]:
res = fsolve(lambda f_n: fn_fix(f_n) - fn_flt(f_n), 1e-5)[0]
res, fn_fix(res)-fn_flt(res)

(0.00352750056054202, -1.864133847284677e-14)

In [12]:
dict_f[2] = res
dict_D[2] = dict_D[1] * D_n(res)

In [13]:
dict_D, dict_f

({0.5: 0.9987515605493182, 1: 0.9970089730807411, 2: 0.9934982362729184},
 {0.5: 0.0024984474705913584,
  1: 0.0034925961006445987,
  2: 0.00352750056054202})

# try for 3 year

In [14]:
def fn_pv_fix(dict_D, tenor, ois_rate):
    """
    return function of PV fixed interms of f_tenor
    """
    # delta_T = 0
    # if tenor > 1:
    #     delta_T
    # for tenor, DF in D.items():
        # # iterate through all DF in D list and return expression
        # print(tenor, DF)
        # y
    # L = (1+f_0/360)^(tenor*360)
    if tenor <2 :
        fn_fix = lambda f_n: ois_rate/((1+f_0/360)**(.5*360)*(1+f_n/360)**(.5*360))
    else:
        sum_df = sum(dict_D.values()) - dict_D[0.5]
        fn_fix = lambda f_n: (sum_df + (1 + f_n / 360)**(360)) * ois_rate
    return fn_fix
fn_fix = fn_pv_fix(dict_D, 3, ois_rates[3])
fn_fix(0.002550494)

0.01002675418106832

In [15]:
def fn_pv_flt(D, tenor, ois_rate,dict_f):
    """
    Return function of PV float in terms of f_tenor
    """
    sum=0
    product = 1

    if tenor <2 :
        fn_flt = lambda  f_1: 1 - 1/((1 + dict_f[0.5] / 360)**(180) * (1 + f_1 / 360)**(180))
    else:
        fn_flt = lambda f_n: dict_D[1]*((1+dict_f[0.5]/360)**180*(1+dict_f[1]/360)**180-1) \
                            + dict_D[2]*((1+dict_f[2]/360)**360-1)\
                            + (dict_D[1]*dict_f[2] * (1+f_n/360)**360)*((1+f_n/360)**360-1)
            
    return fn_flt

# Example usage (assuming ois_rates["6m"] and D are defined)
fn_flt = fn_pv_flt(dict_D, 3, ois_rates[3], dict_f)
fn_flt(0.002550494)


0.006510768039108744

In [16]:
res = fsolve(lambda f_n: fn_fix(f_n) - fn_flt(f_n), 1e-5)[0]
res, fn_fix(res)-fn_flt(res)

(0.6819593215700435, 5.134781488891349e-16)